<a href="https://colab.research.google.com/github/Guest838/ML_Course-3.0/blob/main/05_hw_NeoTokyo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import sklearn

In [ ]:
train=pd.read_csv("drive/MyDrive/ml-task-cosmos/train.csv")
test=pd.read_csv("drive/MyDrive/ml-task-cosmos/test.csv")
train.head()

In [ ]:
test.head()

In [ ]:
train.describe()

In [ ]:
train.isnull().sum()

In [ ]:
train["target"].value_counts()

In [ ]:
corr = np.corrcoef(train)
print(corr)

In [ ]:
y=train['target']
X=train.drop(columns={'target'})


In [ ]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(5)
X = poly.fit_transform(X)

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.1,
    shuffle = True)


In [ ]:
from sklearn.decomposition import PCA

pca = PCA(40)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostClassifier

cat = CatBoostClassifier(iterations=1000,
                           learning_rate= 0.01,
                           loss_function='Logloss',
                           verbose=200).fit(X_train, y_train)

y_pred = cat.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
y_pred.tofile('answers.csv', sep = "\n")

In [ ]:
cat = CatBoostClassifier(iterations=100,
                           loss_function='Logloss',
                           verbose=200)
grid = {'learning_rate': [0.03, 0.01, 0.05],
        'l2_leaf_reg': [1, 3, 5, 7, 9],
        'num_trees': [100, 1000, 10000],
        }

grid_search_result = cat.grid_search(grid,
                                       X=X,
                                       y=y,
                                       plot=True)

In [ ]:
print(grid_search_result)

In [ ]:
import xgboost as xgb

xgb_model = xgb.XGBClassifier(objective="binary:logistic", max_depth = 3,
                              n_estimators=10000, learning_rate=0.001, n_jobs=-1)
xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
import lightgbm as lgb
from sklearn.metrics import classification_report

model = lgb.LGBMClassifier(n_estimators = 10000, learning_rate = 0.001, max_depth=3,
                           objective = "binary", verbosity = -1).fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors = 10).fit(X_train, y_train)
y_pred = neigh.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

rf = RandomForestClassifier(n_estimators = 1000, max_depth=3,
                            ).fit(X_train, y_train)
y_pred = rf.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.ensemble import VotingClassifier

eclf = VotingClassifier(estimators=[
       ('rf', rf), ('cat', cat),("light", model)],
       voting = 'hard').fit(X_train, y_train)
y_pred = eclf.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.ensemble import BaggingClassifier

ccc = BaggingClassifier(estimator=cat,
                        bootstrap_features = True,
                        n_estimators=5).fit(X_train, y_train)
y_pred = ccc.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth=4).fit(X_train, y_train)
y_pred = tree.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn import svm

clf = svm.SVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
import torch
import torchvision
from matplotlib import pyplot as plt
from torch import nn
from torch.nn import functional as F

In [ ]:
CUDA_DEVICE_ID = 0
device = (
    torch.device(f"cuda:{CUDA_DEVICE_ID}") if torch.cuda.is_available() else torch.device("cpu")
)

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(16, 14),
    torch.nn.Dropout(0,5),
    torch.nn.ELU(),
    torch.nn.BatchNorm1d(14),
    torch.nn.Linear(14, 12),
    torch.nn.Dropout(0,5),
    torch.nn.ELU(),
    torch.nn.BatchNorm1d(12),
    torch.nn.Linear(12, 10),
    torch.nn.Dropout(0,5),
    torch.nn.ELU(),
    torch.nn.BatchNorm1d(10),
    torch.nn.Linear(10,8),
    torch.nn.Dropout(0,5),
    torch.nn.ELU(),
    torch.nn.BatchNorm1d(8),
    torch.nn.Linear(8,6),
    torch.nn.Dropout(0,5),
    torch.nn.ELU(),
    torch.nn.BatchNorm1d(6),
    torch.nn.Linear(6, 4),
    torch.nn.Dropout(0,5),
    torch.nn.ELU(),
    torch.nn.BatchNorm1d(4),
    torch.nn.Linear(4, 1),
    torch.nn.Sigmoid()
)

In [ ]:
model.to(device)

In [ ]:
import torch.utils.data as data_utils

X_train = np.array(X_train, dtype = np.float32)
y_train = np.array(y_train, dtype = np.float32)
X_train = torch.from_numpy(X_train)
y_train = torch.from_numpy(y_train)
train_data = data_utils.TensorDataset(X_train, y_train)
train_loader = data_utils.DataLoader(train_data,
                                     batch_size=50, shuffle=True)

X_test = np.array(X_test, dtype = np.float32)
y_test = np.array(y_test, dtype = np.float32)
X_test = torch.from_numpy(X_test)
y_test = torch.from_numpy(y_test)
test_data = data_utils.TensorDataset(X_test, y_test)
test_loader = data_utils.DataLoader(test_data,
                                     batch_size=50, shuffle=True)



In [ ]:
loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)
epochs = 1000
for epoch in range(epochs):
    running_loss = 0
    for inputs, labels in train_loader:
        outputs = model(inputs)
        labels =  labels.unsqueeze(1)
        loss = loss_fn(outputs, labels)


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")

In [ ]:
def get_accuracy(model, data_loader):
    predicted_labels = []
    real_labels = []
    model.eval()
    with torch.no_grad():
        for batch in data_loader:
            y_predicted = model(batch[0].to(device))
            predicted_labels.append(y_predicted.argmax(dim=1).cpu())
            real_labels.append(batch[1])

    predicted_labels = torch.cat(predicted_labels)
    real_labels = torch.cat(real_labels)
    accuracy_score = (predicted_labels == real_labels).type(torch.FloatTensor).mean()
    return accuracy_score

In [ ]:
train_acc_task_1 = get_accuracy(model, train_loader)
print(f"Neural network accuracy on train set: {train_acc_task_1:3.5}")

In [ ]:
test_acc_task_1 = get_accuracy(model, test_loader)
print(f"Neural network accuracy on test set: {test_acc_task_1:3.5}")